In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
# stop_words = set(stopwords.words('english'))

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

!pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("gorkemsevinc/customer_support_tickets", split='train')

In [ ]:
import pandas as pd
df = pd.DataFrame(ds)
df

,Customer Email,Product Purchased,Ticket Type,Ticket Subject,Combined Text,Ticket Priority
0,carrollallison@example.com,gopro hero,technical issue,product setup,i'm having an issue with the gopro hero. pleas...,critical
1,clarkeashley@example.com,lg smart tv,technical issue,peripheral compatibility,i'm having an issue with the lg smart tv. plea...,critical
2,gonzalestracy@example.com,dell xps,technical issue,network problem,i'm facing a problem with my dell xps. the del...,low
3,bradleyolson@example.org,microsoft office,billing inquiry,account access,i'm having an issue with the microsoft office....,low
4,bradleymark@example.com,autodesk autocad,billing inquiry,data loss,i'm having an issue with the autodesk autocad....,low
...,...,...,...,...,...,...
8464,adam28@example.net,lg oled,product inquiry,installation support,my lg oled is making strange noises and not fu...,low
8465,russell68@example.com,bose soundlink speaker,technical issue,refund request,i'm having an issue with the bose soundlink sp...,critical
8466,ashley83@example.org,gopro action camera,technical issue,account access,i'm having an issue with the gopro action came...,high
8467,fpowell@example.org,playstation,product inquiry,payment issue,i'm having an issue with the playstation. plea...,medium


In [ ]:
df = df[['Combined Text', 'Ticket Type']]
df.head()

,Combined Text,Ticket Type
0,i'm having an issue with the gopro hero. pleas...,technical issue
1,i'm having an issue with the lg smart tv. plea...,technical issue
2,i'm facing a problem with my dell xps. the del...,technical issue
3,i'm having an issue with the microsoft office....,billing inquiry
4,i'm having an issue with the autodesk autocad....,billing inquiry


In [ ]:
df['Ticket Type']= df['Ticket Type'].replace({
    'refund request': 'Request',
    'technical issue': 'Problem',
    'cancellation request': 'Request',
    'product inquiry': 'Request',
    'billing inquiry': 'Problem'
})
df.head()

,Combined Text,Ticket Type
0,i'm having an issue with the gopro hero. pleas...,Problem
1,i'm having an issue with the lg smart tv. plea...,Problem
2,i'm facing a problem with my dell xps. the del...,Problem
3,i'm having an issue with the microsoft office....,Problem
4,i'm having an issue with the autodesk autocad....,Problem


In [ ]:
df

,Combined Text,Ticket Type,Processed_Text,category_num
0,i'm having an issue with the gopro hero. pleas...,Problem,issue gopro hero please assist billing zip cod...,0
1,i'm having an issue with the lg smart tv. plea...,Problem,issue lg smart tv please assist need change ex...,0
2,i'm facing a problem with my dell xps. the del...,Problem,face problem dell xps dell xps turn work fine ...,0
3,i'm having an issue with the microsoft office....,Problem,issue microsoft office please assist problem i...,0
4,i'm having an issue with the autodesk autocad....,Problem,issue autodesk autocad please assist note sell...,0
...,...,...,...,...
8464,my lg oled is making strange noises and not fu...,Request,lg ole make strange noise function properly su...,1
8465,i'm having an issue with the bose soundlink sp...,Problem,issue bose soundlink speaker please assist sel...,0
8466,i'm having an issue with the gopro action came...,Problem,issue gopro action camera please assist use di...,0
8467,i'm having an issue with the playstation. plea...,Request,issue playstation please assist not think prod...,1


# Preprocessing

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation, special characters, and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize and lemmatize using spaCy
    stop_words = set(stopwords.words('english'))
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    return " ".join(lemmatized)

In [ ]:
df['Processed_Text'] = df['Combined Text'].apply(preprocess_text)

In [ ]:
df['category_num'] = df['Ticket Type'].map({
    'Problem': 0,
    'Request': 1,

})
df.head()

,Combined Text,Ticket Type,Processed_Text,category_num
0,i'm having an issue with the gopro hero. pleas...,Problem,issue gopro hero please assist billing zip cod...,0
1,i'm having an issue with the lg smart tv. plea...,Problem,issue lg smart tv please assist need change ex...,0
2,i'm facing a problem with my dell xps. the del...,Problem,face problem dell xps dell xps turn work fine ...,0
3,i'm having an issue with the microsoft office....,Problem,issue microsoft office please assist problem i...,0
4,i'm having an issue with the autodesk autocad....,Problem,issue autodesk autocad please assist note sell...,0


In [ ]:
df_x=df['Processed_Text']
df_y=df['category_num']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_x,df_y, test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

In [ ]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
model=LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
y_pred=model.predict(X_test_tfidf)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5619834710743802


In [ ]:
def train_and_evaluate_model(model, model_name):
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy:.4f}\n")
    print(f"{model_name} Classification Report:\n", classification_report(y_test, y_pred))


from sklearn.linear_model import LogisticRegression
train_and_evaluate_model(LogisticRegression(max_iter=1000, random_state=42), "Logistic Regression")

from sklearn.ensemble import RandomForestClassifier
train_and_evaluate_model(RandomForestClassifier(n_estimators=100, random_state=42), "Random Forest Classifier")

from sklearn.svm import SVC
train_and_evaluate_model(SVC(kernel='linear', random_state=42), "Support Vector Machine")


from xgboost import XGBClassifier
train_and_evaluate_model(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42), "XGBoost Classifier")

Logistic Regression Accuracy: 0.5620

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.34      0.06      0.10       706
           1       0.58      0.92      0.71       988

    accuracy                           0.56      1694
   macro avg       0.46      0.49      0.40      1694
weighted avg       0.48      0.56      0.45      1694

Random Forest Classifier Accuracy: 0.5531

Random Forest Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.14      0.21       706
           1       0.58      0.85      0.69       988

    accuracy                           0.55      1694
   macro avg       0.49      0.49      0.45      1694
weighted avg       0.50      0.55      0.49      1694

Support Vector Machine Accuracy: 0.5803

Support Vector Machine Classification Report:
               precision    recall  f1-score   support

           0       0.42      

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:06:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classifier Accuracy: 0.5525

XGBoost Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.16      0.23       706
           1       0.58      0.84      0.69       988

    accuracy                           0.55      1694
   macro avg       0.49      0.50      0.46      1694
weighted avg       0.51      0.55      0.49      1694



# Word2vec

In [ ]:


!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec Model
tokenized_sentences = df['Processed_Text'].apply(lambda x: x.split())  # Tokenize text
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, workers=4)

def word2vec_features(text):
    words = text.split()
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)  # Average word vectors
    else:
        return np.zeros(100)  # If no vectors, return zero vector

X_train_w2v = np.array([word2vec_features(text) for text in X_train])
X_test_w2v = np.array([word2vec_features(text) for text in X_test])

# Combine TF-IDF and Word2Vec features
from scipy.sparse import hstack
X_train_combined = hstack([X_train_tfidf, X_train_w2v])
X_test_combined = hstack([X_test_tfidf, X_test_w2v])


# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_combined, y_train)


Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7fa118dca5c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /usr/local/lib/python3.11/dist-packages/scipy.libs/libscipy_openblas-c128ec02.

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train_smote, y_train_smote)

print("Best parameters:", grid.best_params_)
best_svm = grid.best_estimator_


Best parameters: {'C': 10, 'kernel': 'rbf'}


# XGBoost with Optimized Parameters

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)
xgb.fit(X_train_smote, y_train_smote)
y_pred_xgb = xgb.predict(X_test_combined)

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))


XGBoost Accuracy: 0.5731995277449823


In [ ]:

best_svm = SVC(C=10, kernel='rbf', random_state=42)
best_svm.fit(X_train_smote, y_train_smote)
y_pred_best_svm = best_svm.predict(X_test_combined)

print("SVM (Best parameters) Accuracy:", accuracy_score(y_test, y_pred_best_svm))
print(classification_report(y_test, y_pred_best_svm))


SVM (Best parameters) Accuracy: 0.5289256198347108
              precision    recall  f1-score   support

           0       0.42      0.36      0.39       706
           1       0.59      0.65      0.62       988

    accuracy                           0.53      1694
   macro avg       0.51      0.51      0.50      1694
weighted avg       0.52      0.53      0.52      1694

